# Vehicle Detection Project

The goals / steps of this project are the following:

* Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier
* Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector. 
* Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.
* Implement a sliding-window technique and use your trained classifier to search for vehicles in images.
* Run your pipeline on a video stream and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
* Estimate a bounding box for vehicles detected.

In [ ]:
from glob import glob
import sdcvehicle
from skimage import io
from skimage import feature
from skimage import color

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
test_imgs = [io.imread(f) for f in glob("./test_images/*.jpg")]

## [Rubric](https://review.udacity.com/#!/rubrics/513/view) Points

### Witeup / README

#### 1. Provide a Writeup / README that includes all the rubric points and how you addressed each one.  You can submit your writeup as markdown or pdf.  [Here](https://github.com/udacity/CarND-Vehicle-Detection/blob/master/writeup_template.md) is a template writeup for this project you can use as a guide and a starting point.  

You're reading it! 

### Histogram of Oriented Gradients (HOG)

#### 1. Explain how (and identify where in your code) you extracted HOG features from the training images.

- I am using `skimage.feature.hog` to extract hog features from the vehicle/non-vehicle images. 
- The parameters such as `pixels_per_cell` and `cells_per_block` are tuned as hyperparameters to the pipeline consisting of feature_extraction and classification steps. I will discuss them later on.
- I have applied HOG features on RGB based gray images. I have also tried other color channels such as HSV, but found the differences are not significant.
- The related code are implemented in package `sdcvehicle.classification.ImageFeatExtractor` class. 

In [ ]:
## examples of hog images
extract_hog = lambda img: feature.hog(color.rgb2gray(img), visualise=True,
                pixels_per_cell=(8, 8),
                cells_per_block=(2, 2))
vehicle_img = io.imread("data/vehicles/GTI_Far/image0000.png")
nonvehicle_img = io.imread("data/non-vehicles/GTI/image1.png")
fig, axes = plt.subplots(2, 2, figsize=(6, 6))

axes[0,0].imshow(vehicle_img)
axes[0,1].imshow(extract_hog(vehicle_img)[1], cmap=plt.cm.gray)

axes[1,0].imshow(nonvehicle_img)
axes[1,1].imshow(extract_hog(nonvehicle_img)[1], cmap=plt.cm.gray)


#### 2. Explain how you settled on your final choice of HOG parameters.

- Since the HOG feature extraction is implemented as one step in the vehicle clasifier, its parameter is tuned together with the downstream classifier (SVC) - so I will discuss it in details in the next section.
- This pipeline is implemented in `sdcvehicle.classification.VehicleClassifier` class, in the `__init__()` constructor function.

#### 3. Describe how (and identify where in your code) you trained a classifier using your selected HOG features (and color features if you used them).
- The classifier is implemented in `sdcvehicle.classification.VehicleClassifier` class.
- It consists of two steps
    - `feature_extractor`: it is a combination of HOG features and color HISTOGRAMS., which are implemented in `sdcvehicle.classification.ImageFeatExtractor` class.
    - `svc classifier`: it is a standard LinearSVC from `sklearn` package. It might be more useufl to enable the model to predict posterior probabilities instead of just concrete classes. But here I didn't enable this because it was too time consuming.
    - Hyperparameters, such as `pixels_per_cell`, `cells_per_block` of HOG, `hist_nbins` from color-histogram features, and `C` from SVC, are tuned by a RandomizedSearch based on cross validation (from `sklearn.model_selection.RandomizedSearchCV`. After the optimal values are estimated, they are fixed and used across the whole training process. This is implemented in `sdcvehicle.classification.fit_best_model()`. It is very time consuming.
    - There are two parts of training data:
        - original "vehicle" vs "nonvehicle" from Udacity. I found the data a little noisy, speically in "non-vehicles/Extra" folder, e.g. several images are obviously wrongly labeled: `extra 1864.png`.
        - to further reduce false positives, I augumented the data in two parts:
            - augmented negative examples: random sampling of patches from non-vehicle areas of test_images, it is implemented in `sdcvehicle.classification.enhance_negative_data()`
            - augmented positive examples: random shifting, rotation, and cropping of original vehicle images
        - I found the augumentation useful in reducing both false positive and false negative. But in reality, it could result in overfitting.
    - The final accuracy of trained model on test_data is already over 99%.

In [ ]:
detector = sdcvehicle.detection.VehicleDetector()

### Sliding Window Search

#### 1. Describe how (and identify where in your code) you implemented a sliding window search.  How did you decide what scales to search and how much to overlap windows?

The code of detection is implemented in `sdcvehicle.detection.VehicleDetector` class. I have tried two ways of sampling patches from images.
- Random sampling as in `VehicleDetector.get_random_patches()` method. I found it can give good results but it is more expensive to get comparative results.
- Sliding window + Image Pyramid as in `VEhicleDetector.get_pyramid_slide_window()`: 
    - it iterates through a pyramid of scaled images, so that vehicles with different sizes can be detected with the trained classifier on fixed size images.
    - and then iterates through a sliding with with overlaps, so that images on different locations can be detected.
- The parameters such as sliding window overlap, scaling factor, and start/end space to search, are determined by visual inspection of intermediate results and trial-and-error.

#### 2. Show some examples of test images to demonstrate how your pipeline is working.  What did you do to try to minimize false positives and reliably detect cars?

Vehicle detection in images is implemented in `sdcvehicle.detection.VehicleDetector.detect_in_image()` method. The main steps are,
- sample different patches from image by a sliding window on a pyramid of scaled images.
- predict vehicle/non-vehicle for each patch
- construct a heatmap (based on sum) of predictoin probability based on predictions on each patch
- merge the patch to get bounding boxes from heatmap.

Some results are shown here. From the results we can see that false positives might happen on road shadows, and some boxes are merged into one when two vehicles are too close. But in general the detector does the work.

In [ ]:
fig, axes = plt.subplots(len(test_imgs), 2, figsize=(8*2, 4*len(test_imgs)))
fig.tight_layout()
for test_img, ax in zip(test_imgs, axes):
    plot_img = test_img.copy()
    heatmap = detector.detect_in_image(test_img)
    ax[0].imshow(test_img)
    ax[0].set_axis_off()
    ax[1].imshow(heatmap, cmap=plt.cm.gray)
    ax[1].set_axis_off()

### Video Implementation

#### 1. Provide a link to your final video output.  Your pipeline should perform reasonably well on the entire project video (somewhat wobbly or unstable bounding boxes are ok as long as you are identifying the vehicles most of the time with minimal false positives.)

Here's a [link to my video result](./processed_project_video.mp4). Even though most of the vehicles have been clearly detected. There are still significant amout of false positives - this because the enhanced negative examples are mostly from the test images instead of from videos.

In [ ]:
from moviepy.editor import VideoFileClip
video = VideoFileClip("./project_video.mp4")
processed_video = detector.detect_in_video(video)
processed_video.write_videofile("./output_project_video.mp4", audio=False)

#### 2. Describe how (and identify where in your code) you implemented some kind of filter for false positives and some method for combining overlapping bounding boxes.

As described above, the boxes close enough to each other are merged by following steps (implemented in `sdcvehicle.detection.VehicleDetector.draw_merged_boxes()`)
- generating heatmap based on sum of individual bounding boxes
- thresholding heatmap to generate binary images
- morphology operation such as `opening` to reduce noise
- label the image by `skimage.measure.label`
- find bounding box of each labeled region as detections.

A decay combination of heatmap is also implemented in detection of vehicles in videos (as in `detect_in_video` method), this has the effect that a region that are detected in sequential frames will be boosted to have larger weights in detection.


### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

I can think of a thousand of ways that this method will fail, because it has too many hard coded assumptions even though the classification part is data driven. For example,
- it is very easy to train a classifier with > 95% test accuracy on provided images and yet still generate a lot of false positives. This is just overfitting in disguise.
- using more data or augumentation might help in certain cases, as what I used in the project. but eventually I expect that models with better generalization to work better, e.g., models based on bottleneck features from trained deep learning models (VGG etc). Those features are more general because they are trained on larger dataset, and we can take advantage of this transfer learning effect.
- I notice that most of false positives observed in my experiment are related to the change of gradients, which is a sign that HOG features might overfit in deciding that a patch is a vehicle. It might be caused by the lack of complicated textures in the negative training examples. It might help if other models are used, which can output a probability instead of a hard class label, e.g., RandomForest.
- In general the model implemented here is too slow for real-world detection and very easy to overfit. But I have learned the basics through it.